<a href="https://colab.research.google.com/github/agathalafaiety/petlens-genai/blob/main/PetLens.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai

In [ ]:
!pip install -q google-adk

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Pergunta ao Gemini uma informação mais recente que seu conhecimento

from IPython.display import HTML, Markdown

# Perguntar pro modelo quando é a próxima imersão de IA ###############################################
resposta = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {resposta.text}"))

Resposta:
 A Alura não tem uma data fixa para a Imersão IA com Google Gemini. A melhor forma de saber sobre futuras edições é:

*   **Acompanhar a página oficial da Alura:** Fique de olho na página da Alura (alura.com.br) e procure por eventos ou imersões futuras.
*   **Seguir as redes sociais da Alura:** A Alura geralmente anuncia novas imersões e cursos em suas redes sociais (LinkedIn, Instagram, etc.).
*   **Assinar a newsletter da Alura:** Ao se inscrever na newsletter, você receberá informações sobre novos cursos, eventos e promoções.

In [ ]:
# Pergunta ao Gemini uma informação utilizando a busca do Google como contexto

response = client.models.generate_content(
    model=MODEL_ID,
    contents='Quando é a próxima Imersão IA com Google Gemini da Alura?',
    config={"tools": [{"google_search": {}}]}
    # Inserir a tool de busca do Google ###############################################
)

# Exibe a resposta na tela
display(Markdown(f"Resposta:\n {response.text}"))

Resposta:
 A próxima Imersão IA com Google Gemini da Alura aconteceu de 12 a 16 de maio de 2025. As inscrições para esta edição encerraram em 11 de maio de 2025.

In [ ]:
# Exibe a busca
print(f"Busca realizada: {response.candidates[0].grounding_metadata.web_search_queries}")
# Exibe as URLs nas quais ele se baseou
print(f"Páginas utilizadas na resposta: {', '.join([site.web.title for site in response.candidates[0].grounding_metadata.grounding_chunks])}")
print()
display(HTML(response.candidates[0].grounding_metadata.search_entry_point.rendered_content))

Busca realizada: ['próxima Imersão IA com Google Gemini Alura', 'Alura Imersão IA Google Gemini datas']
Páginas utilizadas na resposta: thallesbenicio.com.br, eucapacito.com.br, convergenciadigital.com.br, tecmundo.com.br, alura.com.br



In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
###################################################
# --- Agente Pet 1: Diagnóstico Auxiliado por IA --- #
###################################################
def agente_diagnostico_pet(sintomas_descritos: str):
    diagnostico_agent = Agent(
        name="agente_diagnostico_pet",
        model="gemini-2.0-flash",
        instruction="""Você é um assistente de IA especializado em saúde animal, projetado para fornecer orientação preliminar a donos de pets preocupados.
Analise os sintomas do pet que serão fornecidos na entrada de dados.

Com base EXCLUSIVAMENTE nos sintomas fornecidos na entrada, você deve:
1.  Gerar uma lista de 2 a 4 possíveis diagnósticos diferenciais. Para cada um, explique brevemente a relação com os sintomas descritos. Deixe absolutamente claro que estas são apenas hipóteses iniciais e NÃO constituem um diagnóstico definitivo.
2.  Enfatizar FORTEMENTE que suas sugestões são apenas para orientação preliminar e NÃO substituem a avaliação, o diagnóstico e o tratamento por um médico veterinário qualificado.
3.  Recomendar SEMPRE que o dono do pet procure um médico veterinário.
4.  Avaliar e comunicar a urgência da situação com base nos sintomas (ex: 'emergência imediata', 'procurar veterinário hoje', 'agendar nos próximos dias', 'observar e procurar veterinário se persistir/piorar').
5.  NÃO prescrever medicamentos ou tratamentos.
6.  Se julgar estritamente necessário para entender um sintoma raro/complexo para formular hipóteses, você pode indicar a necessidade de usar a ferramenta Google Search (se disponível), mas sua resposta principal deve ser baseada no seu conhecimento treinado e nas informações fornecidas.

Lembre-se: o objetivo é ajudar o dono a entender a potencial gravidade e a importância de buscar ajuda profissional, não diagnosticar.
Priorize sempre o bem-estar do animal.""",
        description="Agente que fornece orientação preliminar sobre sintomas de pets, possíveis diagnósticos diferenciais e recomendação veterinária.",
        tools=[google_search],
    )
    entrada_do_agente = f"Sintomas descritos pelo tutor: {sintomas_descritos}"
    orientacao = call_agent(diagnostico_agent, entrada_do_agente)
    return orientacao

In [ ]:
############################################################
# --- Agente Pet 2: Personalização de Cuidados Preventivos --- #
############################################################
def agente_cuidados_preventivos(raca: str, idade: int, historico_medico: str, localizacao_geografica: str):
    cuidados_agent = Agent(
        name="agente_cuidados_preventivos",
        model="gemini-2.0-flash",
        instruction="""Você é um assistente de IA especializado em saúde preventiva para pets.
Com base nas informações do pet fornecidas na entrada (raça, idade, histórico médico, localização geográfica), sua tarefa é gerar um plano de cuidados preventivos personalizado.
O plano deve incluir sugestões para:
1.  Vacinação (considerando idade, estilo de vida e riscos locais).
2.  Vermifugação (frequência e tipo).
3.  Exames de rotina e específicos (com base na raça, idade e predisposições).
4.  Controle de parasitas externos (pulgas, carrapatos), considerando a localização.
5.  Recomendações dietéticas básicas ou pontos de atenção nutricional, se aplicável.
Use a ferramenta Google Search, se necessário, para pesquisar diretrizes de vacinação atualizadas, riscos de doenças endêmicas na localização fornecida ou predisposições de raças específicas.
Sempre enfatize que este plano é uma sugestão e deve ser discutido e validado por um médico veterinário.
""",
        description="Agente que gera um plano de cuidados preventivos personalizado para pets.",
        tools=[google_search],
    )
    entrada_do_agente = (
        f"Dados do Pet para Plano Preventivo:\n"
        f"- Raça: {raca}\n"
        f"- Idade: {idade} anos\n"
        f"- Histórico Médico: {historico_medico}\n"
        f"- Localização Geográfica: {localizacao_geografica}"
    )
    plano_preventivo = call_agent(cuidados_agent, entrada_do_agente)
    return plano_preventivo

In [ ]:
################################################################
# --- Agente Pet 3: Enriquecimento Ambiental Personalizado --- #
################################################################
def agente_enriquecimento_ambiental(raca: str, idade: int, nivel_energia: str, historico_comportamento: str):
    enriquecimento_agent = Agent(
        name="agente_enriquecimento_ambiental",
        model="gemini-2.0-flash",
        instruction="""Você é um especialista em comportamento e bem-estar animal, focado em enriquecimento ambiental para pets.
Com base nas informações do pet fornecidas na entrada (raça, idade, nível de energia, histórico de comportamento), sua tarefa é sugerir de 3 a 5 ideias criativas e personalizadas de enriquecimento ambiental.
Para cada sugestão, explique:
1.  Como ela atende às necessidades específicas do pet (considerando raça, idade, energia).
2.  Quais benefícios ela traz para o bem-estar físico e mental do animal.
3.  Como implementá-la de forma segura.
As sugestões podem incluir brinquedos, atividades, modificações no ambiente doméstico, ou técnicas de treinamento positivo.
Você pode usar a ferramenta Google Search para buscar ideias inovadoras ou específicas para certas raças/comportamentos, mas adapte-as ao perfil do pet.
O objetivo é fornecer sugestões práticas que os tutores possam implementar.
""",
        description="Agente que sugere ideias personalizadas de enriquecimento ambiental para pets.",
        tools=[google_search],
    )
    entrada_do_agente = (
        f"Dados do Pet para Sugestões de Enriquecimento Ambiental:\n"
        f"- Raça: {raca}\n"
        f"- Idade: {idade} anos\n"
        f"- Nível de Energia: {nivel_energia}\n"
        f"- Histórico de Comportamento: {historico_comportamento}"
    )
    sugestoes_enriquecimento = call_agent(enriquecimento_agent, entrada_do_agente)
    return sugestoes_enriquecimento

In [ ]:
######################################################################
# --- Agente Pet 4: Localização Inteligente de Serviços Pet-Friendly --- #
#######################################################################
def agente_servicos_pet_friendly(necessidade_especifica: str, localizacao_usuario: str):
    servicos_agent = Agent(
        name="agente_servicos_pet_friendly",
        model="gemini-2.0-flash",
        instruction="""Você é um assistente especializado em encontrar serviços pet-friendly.
Sua tarefa é entender a necessidade específica do usuário e a localização fornecida na entrada para encontrar de 2 a 3 opções altamente relevantes.
Utilize a ferramenta Google Search de forma eficaz para realizar a busca.
Para cada opção encontrada, forneça:
1.  Nome do estabelecimento/serviço.
2.  Endereço (se aplicável e encontrado).
3.  Contato (telefone ou website, se encontrado).
4.  Uma breve descrição destacando por que é relevante para a necessidade do usuário (ex: "atendimento emergencial 24h para gatos", "ampla área verde cercada para cães grandes", "especialista em dermatologia canina").
5.  Informações adicionais úteis, como horários de funcionamento ou faixas de preço, se facilmente disponíveis na busca.
Priorize resultados que pareçam mais confiáveis ou bem avaliados, se essa informação surgir.
Formate a resposta de maneira clara e fácil de ler.
""",
        description="Agente que localiza serviços pet-friendly com base nas necessidades do usuário.",
        tools=[google_search],
    )
    entrada_do_agente = (
        f"Necessidade do Usuário: {necessidade_especifica}\n"
        f"Localização do Usuário (ou área de busca): {localizacao_usuario}"
    )
    locais_encontrados = call_agent(servicos_agent, entrada_do_agente)
    return locais_encontrados


In [ ]:
if __name__ == '__main__':
    data_hoje_formatada = date.today().strftime("%d/%m/%Y")

    print("🐾 Bem-vindo ao Sistema de Assistência Pet com IA 🐾")
    print("====================================================\n")

    # Agente de Diagnóstico
    print("--- Testando Agente de Diagnóstico ---")
    sintomas = input("❓ Descreva os sintomas do seu pet: ")
    if sintomas:
        orientacao_diagnostica = agente_diagnostico_pet(sintomas)
        print(f"\n--- 📝 Resultado do Agente de Diagnóstico ---\n{orientacao_diagnostica}")
        print("--------------------------------------------------------------")
    else:
        print("Nenhum sintoma fornecido.")

    # Agente de Cuidados Preventivos
    print("\n--- Testando Agente de Cuidados Preventivos ---")
    raca_pet = input("❓ Raça do pet: ")
    idade_pet_str = input("❓ Idade do pet (anos): ")
    try:
        idade_pet = int(idade_pet_str)
        hist_med = input("❓ Breve histórico médico (ex: 'saudável', 'alergias conhecidas'): ")
        loc_geo = input("❓ Cidade/Estado onde o pet vive: ")
        if raca_pet and loc_geo:
            plano = agente_cuidados_preventivos(raca_pet, idade_pet, hist_med, loc_geo)
            # display(to_markdown(plano))
            print(f"\n--- 📝 Resultado do Agente de Cuidados Preventivos ---\n{plano}")
            print("--------------------------------------------------------------")
        else:
            print("Informações insuficientes para o plano preventivo.")
    except ValueError:
        print("Idade inválida.")


    # Agente de Enriquecimento Ambiental
    print("\n--- Testando Agente de Enriquecimento Ambiental ---")
    raca_enr = input("❓ Raça do pet para enriquecimento: ")
    idade_enr_str = input("❓ Idade do pet (anos): ")
    try:
        idade_enr = int(idade_enr_str)
        energia_pet = input("❓ Nível de energia (ex: baixo, médio, alto): ")
        comportamento_pet = input("❓ Breve histórico de comportamento (ex: 'ansioso', 'adora caçar bolinhas'): ")
        if raca_enr and energia_pet:
            sugestoes = agente_enriquecimento_ambiental(raca_enr, idade_enr, energia_pet, comportamento_pet)
            # display(to_markdown(sugestoes))
            print(f"\n--- 📝 Resultado do Agente de Enriquecimento Ambiental ---\n{sugestoes}")
            print("--------------------------------------------------------------")
        else:
            print("Informações insuficientes para sugestões de enriquecimento.")
    except ValueError:
        print("Idade inválida.")

    # Agente de Serviços Pet-Friendly
    print("\n--- Testando Agente de Serviços Pet-Friendly ---")
    necessidade = input("❓ Qual serviço pet-friendly você procura? (ex: 'veterinário emergência para gatos'): ")
    local = input("❓ Perto de qual localização? (ex: 'Bairro Pinheiros, São Paulo'): ")
    if necessidade and local:
        servicos = agente_servicos_pet_friendly(necessidade, local)
        # display(to_markdown(servicos))
        print(f"\n--- 📝 Resultado do Agente de Serviços Pet-Friendly ---\n{servicos}")
        print("--------------------------------------------------------------")
    else:
        print("Informações insuficientes para buscar serviços.")